# GloFAS historical

In [1]:
import climetlab as cml
import xpublish
import xarray as xr
# import rioxarray
import numpy as np
from xpublish.utils.ows import Render
from xpublish.routers import XYZFactory
from datashader import transfer_functions as tf
import datashader as ds
import colorcet
import numpy as np
import nest_asyncio 
nest_asyncio.apply()

In [2]:
hist = cml.load_dataset(
            'cems-flood-glofas-historical',
            model='lisflood',
            product_type='intermediate',
            system_version='version_3_1',
            period= '20210710',
            variable="river_discharge_in_the_last_24_hours",
            #area= nwse,
            #split_on = ['day'],
            #threads = 6,
            #merger=False
        )

By downloading data from this dataset, you agree to the terms and conditions defined at https://github.com/ecmwf-lab/climetlab_cems_flood/LICENSEIf you do not agree with such terms, do not download the data. 


In [3]:
dataset = hist.to_xarray()

In [4]:
dataset = dataset.isel(time=0)

In [9]:
var = "dis24"
#var = "air"

crs_epsg =  4326

class DataShader(Render):
    def __init__(self, aggregation={}, color_mapping={}):
        super().__init__(aggregation=aggregation, color_mapping=color_mapping)

    def aggregation(self, tile):
        self.cvs = ds.Canvas(plot_width=256, plot_height=256)
        agg = self.cvs.raster(tile, **self.agg_params)
        return agg

    def color_mapping(self, agg):
        img = tf.shade(agg, **self.cm_params)
        return img


render = DataShader(
                      aggregation={"agg":"mean","interpolate":"nearest"}, 
                      color_mapping={ 
                                "cmap": colorcet.kbc,
                                "how": "log", 
                                "span": [float(np.nanmin(dataset[var].values)), 
                                         float(np.nanmax(dataset[var].values))], 
                                "alpha": 100 }
                        )


xyz_router = XYZFactory(
                       crs_epsg= crs_epsg,
                       render= render,
                    #    transformers=[transform2]
)

In [10]:
rest = xpublish.Rest(dataset, routers=[xyz_router])

In [11]:
rest.serve()

INFO:     Started server process [641421]
2022-05-28 05:28:20,977 INFO Started server process [641421]
INFO:     Waiting for application startup.
2022-05-28 05:28:20,978 INFO Waiting for application startup.
INFO:     Application startup complete.
2022-05-28 05:28:20,980 INFO Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9000 (Press CTRL+C to quit)
2022-05-28 05:28:20,981 INFO Uvicorn running on http://0.0.0.0:9000 (Press CTRL+C to quit)


INFO:     127.0.0.1:46414 - "GET /tiles/dis24/6/44/31?xlab=longitude&ylab=latitude HTTP/1.1" 200 OK
INFO:     127.0.0.1:46410 - "GET /tiles/dis24/6/46/32?xlab=longitude&ylab=latitude HTTP/1.1" 200 OK
INFO:     127.0.0.1:46404 - "GET /tiles/dis24/6/45/32?xlab=longitude&ylab=latitude HTTP/1.1" 200 OK
INFO:     127.0.0.1:46408 - "GET /tiles/dis24/6/44/32?xlab=longitude&ylab=latitude HTTP/1.1" 200 OK
INFO:     127.0.0.1:46412 - "GET /tiles/dis24/6/45/33?xlab=longitude&ylab=latitude HTTP/1.1" 200 OK
INFO:     127.0.0.1:46406 - "GET /tiles/dis24/6/45/31?xlab=longitude&ylab=latitude HTTP/1.1" 200 OK
INFO:     127.0.0.1:46414 - "GET /tiles/dis24/6/46/31?xlab=longitude&ylab=latitude HTTP/1.1" 200 OK
INFO:     127.0.0.1:46410 - "GET /tiles/dis24/6/44/33?xlab=longitude&ylab=latitude HTTP/1.1" 200 OK
INFO:     127.0.0.1:46404 - "GET /tiles/dis24/6/46/33?xlab=longitude&ylab=latitude HTTP/1.1" 200 OK
INFO:     127.0.0.1:46412 - "GET /tiles/dis24/6/47/32?xlab=longitude&ylab=latitude HTTP/1.1" 200 OK


INFO:     Shutting down
2022-05-28 05:32:09,887 INFO Shutting down
INFO:     Waiting for application shutdown.
2022-05-28 05:32:09,988 INFO Waiting for application shutdown.
INFO:     Application shutdown complete.
2022-05-28 05:32:09,989 INFO Application shutdown complete.
INFO:     Finished server process [641421]
2022-05-28 05:32:09,990 INFO Finished server process [641421]
